In [3]:
import requests
import os
import sys
import pandas as pd

NEWS_API_KEY = os.getenv('NEWS_API_KEY')
current_dir = os.path.dirname(os.path.abspath('__file__'))
plugins_dir = os.path.join(current_dir, '../plugins')
sys.path.append(plugins_dir)

In [4]:
from operators.NewsAPIOperator import NewsAPIToDataframeOperator

operator = NewsAPIToDataframeOperator(
task_id="extract_api_data",
news_topic='bitcoin',
endpoint = "top-headlines",
from_date = '2024-05-18',
to_date = None,
sort_by = "popularity", 
page_size = 100, 
page_number = 1)

dataframe = operator.execute()


[2024-06-01T17:41:11.656-0400] {baseoperator.py:399} WARNING - NewsAPIToDataframeOperator.execute cannot be called outside TaskInstance!
{'status': 'ok', 'totalResults': 4839, 'articles': [{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'Joel Khalili', 'title': 'Craig Wright Lied About Creating Bitcoin And Faked Evidence, Judge Rules', 'description': 'A UK judge has determined that Craig Wright forged evidence in a campaign to prove he is Satoshi Nakamoto, creator of Bitcoin, in a move that prevents him from bringing further lawsuits in the country.', 'url': 'https://www.wired.com/story/craig-wright-lied-faked-evidence-bitcoin-judge-says/', 'urlToImage': 'https://media.wired.com/photos/663964ab2a04ee031b4d1808/191:100/w_1280,c_limit/Craig-Wright-Sentencing-Business-2024583568.jpg', 'publishedAt': '2024-05-20T11:01:15Z', 'content': 'Weve seen a cascading effect from the pronouncement on a host of other litigations globally, says Grewal. For people outside of crypto, [all this] mi

In [5]:
print (dataframe.iloc[0])
df = dataframe.copy()
print(df.iloc[0])

source                          {'id': 'wired', 'name': 'Wired'}
author                                              Joel Khalili
title          Craig Wright Lied About Creating Bitcoin And F...
description    A UK judge has determined that Craig Wright fo...
url            https://www.wired.com/story/craig-wright-lied-...
urlToImage     https://media.wired.com/photos/663964ab2a04ee0...
publishedAt                                 2024-05-20T11:01:15Z
content        Weve seen a cascading effect from the pronounc...
Name: 0, dtype: object
source                          {'id': 'wired', 'name': 'Wired'}
author                                              Joel Khalili
title          Craig Wright Lied About Creating Bitcoin And F...
description    A UK judge has determined that Craig Wright fo...
url            https://www.wired.com/story/craig-wright-lied-...
urlToImage     https://media.wired.com/photos/663964ab2a04ee0...
publishedAt                                 2024-05-20T11:01:15Z
co

In [6]:
from scripts.news.transform_news import transform_news_bronze
import pandas as pd
df = dataframe.copy()
bronze = transform_news_bronze(df, 'bitcoin', 100)
print(bronze.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   source        100 non-null    object             
 1   author        94 non-null     object             
 2   title         100 non-null    object             
 3   description   99 non-null     object             
 4   url           100 non-null    object             
 5   published_at  100 non-null    datetime64[ns, UTC]
 6   content       100 non-null    object             
 7   job_log_id    100 non-null    int64              
 8   topic         100 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(7)
memory usage: 7.2+ KB
None


In [7]:
df = bronze.copy()
from transformers import pipeline

MIN_WORD_COUNT_CONTENT = 10
def transform_news_silver(df):
    # Basic Transformations
    df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
    df['published_date'] = df['published_at'].dt.date.astype('datetime64[s]')
    df["year"] = df["published_date"].dt.year
    df["month"] = df["published_date"].dt.month
    df["day"] = df["published_date"].dt.day

    df = df[df['content'].apply(lambda x: len(x.split(' ')) >= MIN_WORD_COUNT_CONTENT)]
    
    # Sentiment Analysis
    sentiment_pipeline = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', tokenizer='mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')
    def get_sentiment(text):
        result = sentiment_pipeline(text)
        return {'label': result[0]['label'], "score": result[0]['score']}
    
    df['title_sentiment'] = df['title'].apply(lambda x: get_sentiment(x).get('label'))
    df['title_score'] = df['title'].apply(lambda x: get_sentiment(x).get('score'))

    df['content_sentiment'] = df['content'].apply(lambda x: get_sentiment(x).get('label'))
    df['content_score'] = df['content'].apply(lambda x: get_sentiment(x).get('score'))

    # Drop duplicates
    df.drop_duplicates(inplace=True)
    return df 

silver = transform_news_silver(df)

In [8]:
from scripts.news.transform_news_spark import transform_news_silver_spark

df = bronze.copy()
silver = transform_news_silver_spark(df)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/calvindu/School/Projects/Investment_Data_Analysis/env/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.